In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from scripts.ml_method import *
from scripts.proj1_helpers import *
from scripts.preprocess import *
from scripts.split_data import *
from scripts.model_testing import *
from scripts.cross_validation import *

%load_ext autoreload
%autoreload 2

In [3]:
# load data
y, x, ind = load_csv_data('higgs-data/train.csv')

In [4]:
def getPara(x, y):
    for i in range(4):
        mask = x[:, 22] == i
        x_tmp = x[mask, :]
        y_tmp = y[mask]

        if i == 0:
            x_tmp = np.delete(x_tmp,[4,5,6,11,12,15,18,20,22,23,24,25,26,27,28,29],1)
        elif i == 1:
            x_tmp = np.delete(x_tmp,[4,5,6,11,12,15,18,20,22,26,27,28],1)
        else:
            x_tmp = np.delete(x_tmp, [11,15,18,20,22,28],1)

        stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)

        losses, w = reg_logistic_regression(y_tmp, stand_x, np.zeros(stand_x.shape[1]), 5000, 0.000002, 0)
        print(np.mean(predict_labels(w, stand_x) == y_tmp))
        
#         k_fold = 5
#         acc_tr = np.zeros(k_fold)
#         acc_te = np.zeros(k_fold)
#         weights = np.zeros([k_fold, stand_x.shape[1]])

#         k_indices = build_k_indices(y_tmp, k_fold, 1)

#         for k in range(k_fold):
#             acc_tr[k], acc_te[k], weights[k,:], losses = cross_validation(y_tmp, stand_x, k_indices, k, 2000, 0)
#             print(acc_tr[k], ', ', acc_te[k], ', ', losses)

        if i == 0:
            w0 = w
#             w0 = np.zeros(stand_x.shape[1])
#             for j in range(stand_x.shape[1]):
#                 w0[j] = sum(weights[:,j])/k_fold
        if i == 1:
            w1 = w
#             w1 = np.zeros(stand_x.shape[1])
#             for j in range(stand_x.shape[1]):
#                 w1[j] = sum(weights[:,j])/k_fold
        if i == 2:
            w2 = w
#             w2 = np.zeros(stand_x.shape[1])
#             for j in range(stand_x.shape[1]):
#                 w2[j] = sum(weights[:,j])/k_fold
        if i == 3:
            w3 = w
#             w3 = np.zeros(stand_x.shape[1])
#             for j in range(stand_x.shape[1]):
#                 w3[j] = sum(weights[:,j])/k_fold
    return w0, w1, w2, w3

w0, w1, w2, w3 = getPara(x, y)

KeyboardInterrupt: 

In [10]:
y, x, ind = load_csv_data('higgs-data/test.csv')

y_pred = np.ones(len(y))

mask = x[:, 22] == 0
x_tmp = x[mask, :]
x_tmp = np.delete(x_tmp,[4,5,6,11,12,15,18,20,22,23,24,25,26,27,28,29],1)
stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)
y_pred[mask] = predict_labels(w0, stand_x)

mask = x[:, 22] == 1
x_tmp = x[mask, :]
x_tmp = np.delete(x_tmp,[4,5,6,11,12,15,18,20,22,25,26,27,28,29],1)
stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)
y_pred[mask] = predict_labels(w1, stand_x)

mask = x[:, 22] == 2
x_tmp = x[mask, :]
x_tmp = np.delete(x_tmp, [11,15,18,20,22,28],1)
stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)
y_pred[mask] = predict_labels(w2, stand_x)

mask = x[:, 22] == 3
x_tmp = x[mask, :]
x_tmp = np.delete(x_tmp, [11,15,18,20,22,28],1)
stand_x = standardize_with_power_terms(x_tmp, 2, True, with_sqrt=True)
y_pred[mask] = predict_labels(w3, stand_x)

create_csv_submission(ind, y_pred, 'prediction.csv')

## ------ below is just used to test -----

In [71]:
i = 1
mask = x[:, 22] == i
x_tmp = x[mask, :]
y_tmp = y[mask]

phi_list = [11,15,18,20]
# phi_list = [11, 15, 18, 20]


if i == 0:
    for i in phi_list:
        x_tmp1 = np.cos(x_tmp[:,i])
        x_tmp = np.concatenate((x_tmp, x_tmp1.reshape([len(x_tmp1),1])),1)
        x_tmp1 = np.sin(x_tmp[:,i])
        x_tmp = np.concatenate((x_tmp, x_tmp1.reshape([len(x_tmp1),1])),1)    
#     x_tmp = np.delete(x_tmp, [4,5,6,12,22,23,24,25,26,27,28,29], 1)
    x_tmp = np.delete(x_tmp,[4,5,6,11,12,15,18,20,22,23,24,25,26,27,28,29],1)
elif i == 1:
    for i in phi_list:
        x_tmp1 = np.cos(x_tmp[:,i])
        x_tmp = np.concatenate((x_tmp, x_tmp1.reshape([len(x_tmp1),1])),1)
        x_tmp1 = np.sin(x_tmp[:,i])
        x_tmp = np.concatenate((x_tmp, x_tmp1.reshape([len(x_tmp1),1])),1)  
    x_tmp = np.delete(x_tmp,[4,5,6,11,12,15,18,20,22,26,27,28],1)
# else:
#     x_tmp = np.delete(x_tmp, [11,15,18,20,22,28],1)

# x_tmp = np.delete(x_tmp,[4,5,6,11,12,15,18,20,22,23,24,25,26,27,28,29],1)
stand_x = standardize_with_power_terms(x_tmp, 4, True, with_sqrt=True)
# x_tr, x_te, y_tr, y_te = split_data(stand_x, y_tmp, 0.8, myseed=1)

# for lam in range(100):
#     weights = ridge_regression(y_tr, x_tr, lam * 0.1)
#     y_pred =  predict_labels(weights, x_te)
#     print(np.mean(y_pred == y_te))

for lam in range(100):
    weights = ridge_regression(y_tmp, stand_x, lam * 0.1)
    y_pred =  predict_labels(weights, stand_x)
    print(np.mean(y_pred == y_tmp))

0.787230475601
0.757982564737
0.751457237181
0.747885071701
0.744841638296
0.741308160528
0.738509749304
0.736859073558
0.734847312494
0.733441658929
0.732281027546
0.730939853503
0.729585783555
0.728412256267
0.727535334778
0.726619725575
0.725768595894
0.725304343341
0.724388734138
0.723627875787
0.722815433818
0.722248014031
0.721616114722
0.7209455277
0.720042814402
0.719565665944
0.718791911689
0.718172908284
0.717824718869
0.717424945837
0.717025172805
0.71676725472
0.716599607965
0.716341689879
0.71616114722
0.715967708656
0.715851645517
0.715619519241
0.715206850304
0.715039203549
0.714819973177
0.714613638708
0.714523367379
0.714291241102
0.714188073868
0.7139817394
0.71380119674
0.713517486846
0.713336944187
0.713220881048
0.713053234293
0.71301454658
0.712743732591
0.71240843908
0.712395543175
0.712343959558
0.712150520995
0.711841019292
0.711750747962
0.711570205303
0.71144124626
0.711312287218
0.711234911792
0.711093056845
0.710976993707
0.710706179717
0.710667492005
0.710